In [1]:
import torch
import numpy as np
from utils import get_freest_gpu
import torch.nn.functional as F

In [2]:
from predict_transformer_nonlinear import NonLinearNet

In [3]:
freest_gpu = get_freest_gpu()
freest_gpu

1

In [4]:
torch.cuda.set_device(f'cuda:{freest_gpu}')

In [33]:
net = NonLinearNet(3, 3, 64, 64, mlp=True).cuda()

In [34]:
net.load_state_dict(torch.load('proper_split_mlp.parameters'))

RuntimeError: Error(s) in loading state_dict for NonLinearNet:
	Missing key(s) in state_dict: "reward_layer.linear1.weight", "reward_layer.linear1.bias", "reward_layer.linear2.weight", "reward_layer.linear2.bias". 

In [7]:
traj_encoder = net.trajectory_encoder
state_encoder = net.state_encoder

In [8]:
states = torch.Tensor(np.load('states_46000.npy')).to(torch.int64)
states = F.one_hot(states, num_classes=4).view(-1, 150, 100).to(torch.float)

In [14]:
# states_unsqueezed = states.unsqueeze(0)

In [30]:
traj = states[10002]
traj = traj.unsqueeze(0).cuda()

In [31]:
reward_est = net(traj, traj[0])
reward_est

tensor([ 0.2230,  0.2037,  0.2037, -0.1705,  0.2037, -0.1705,  0.2037,  0.2037,
        -0.1705,  0.2037,  0.2037, -0.1705,  0.2037,  0.2037,  0.2037,  0.2037,
        -0.1705,  0.2037,  0.2230,  0.2037,  0.2037, -0.1705,  0.2037,  0.2037,
         0.2037, -0.1705,  0.2037, -0.1705,  0.2037,  0.2037, -0.1705,  0.2037,
        -0.1705,  0.2037,  0.0478, -0.0335,  0.4984, -0.0335,  0.2230,  0.2037,
         0.2037,  0.2230,  0.2037,  0.2037,  0.0478, -0.0335,  0.0478, -0.0335,
         0.0478,  0.0478, -0.0335,  0.0478, -0.0335,  0.0478,  0.0478, -0.0335,
         0.0478, -0.0335,  0.0478, -0.0335,  0.2230,  0.2037,  0.2230,  0.2037,
         0.2230,  0.2037,  0.0478, -0.0335,  0.0478, -0.0335,  0.0478,  0.0478,
         0.0478, -0.0335,  0.0478, -0.0335,  0.0478, -0.0335,  0.0478, -0.0335,
         0.0478, -0.0335,  0.0478, -0.0335,  0.0478, -0.0335,  0.0478, -0.0335,
        -0.0335,  0.0478, -0.0335,  0.0478, -0.0335,  0.0478,  0.0478, -0.0335,
         0.2230,  0.2037, -0.1705,  0.20

In [15]:
state_rep = state_encoder(traj[0][0])
state_rep

tensor([ 0.2969, -0.0962, -0.2074], device='cuda:1', grad_fn=<AddBackward0>)

In [16]:
traj[0][0].view(5, 5, 4)

tensor([[[1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.]],

        [[0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.]],

        [[0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.]],

        [[1., 0., 0., 0.],
         [0., 0., 0., 1.],
         [1., 0., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.]],

        [[0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [1., 0., 0., 0.],
         [0., 0., 0., 1.]]], device='cuda:1')

In [27]:
traj_rep = traj_encoder(traj)
traj_rep

tensor([[1., 1., 1.]], device='cuda:1', grad_fn=<SigmoidBackward0>)

In [32]:
reward_layer = net.reward_layer

AttributeError: 'NonLinearNet' object has no attribute 'reward_layer'